# Import tools

In [ ]:
#! pip3 install torch torchvision torchaudio
#conda install pytorch torchvision -c pytorch

In [ ]:
# Additional Dependencies
!pip install barbar torchsummary 

In [ ]:
#test 
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
# helpful packages to load
import time
import copy
import pickle
from barbar import Bar
import numpy as np 
import pandas as pd 
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
%matplotlib inline

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchsummary import summary

from tqdm import tqdm
from pathlib import Path
import gc
RANDOMSTATE = 0
import os

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
pwd

In [ ]:
path = "./clean_data.csv" #directory for data
df = pd.read_csv(path)
df.info()
df

# Process data

In [ ]:
# preparing intermediate DataFrame
path = Path('./Data/N_train_images_model/')
train_df  = pd.DataFrame()
train_df['image'] = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
train_df['image'] = './Data/N_train_images_model/' + train_df['image'].astype(str)

#delete incomplete image files that cannot be opened
def delete_wrong_dim(df):
    wrong_dim = []
    m = 0
    for i in df['image']:
            m += 1
            with open(i, 'rb') as f:
                check_chars = f.read()[-2:]
            if check_chars != b'\xff\xd9':
                print('Not complete image')
                x = df[df['image']== i].index.values[0]
                wrong_dim.append(x)
                print(x)
        
    df.info()
    df = df.drop(wrong_dim)
    df.info()
    return df

def add_class(inp_df):
    inp_df['id'] = inp_df.image.str.split("/", expand = True)[3].str.strip('.jpg')
    inp_df['landmark_id'] = inp_df['id'].map(df.set_index(['id'])['landmark_id'])
    return inp_df

train_df = delete_wrong_dim(train_df)


In [ ]:
train_df = add_class(train_df)
train_df = train_df[['image', 'landmark_id']]
train_df.landmark_id = train_df['landmark_id'].astype('category')
from sklearn import preprocessing
# Let's try again with labels
label_encoder = preprocessing.LabelEncoder()
train_df['landmark_id'] = label_encoder.fit_transform(train_df['landmark_id'])
y_train = train_df['landmark_id']
y_train
train_df = train_df[['image']]
train_df

In [ ]:
path = Path('./Data/M_validation_images_model/')
val_df  = pd.DataFrame()
val_df['image'] = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
val_df['image'] = './Data/M_validation_images_model/' + val_df['image'].astype(str)

val_df.head()
val_df = delete_wrong_dim(val_df)
#val_df = add_class(val_df)

In [ ]:
val_df.info()

In [ ]:
path = Path('./Data/N_test_images_from_train/')
test_df  = pd.DataFrame()
test_df['image'] = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
test_df['image'] = './Data/N_test_images_from_train/' + test_df['image'].astype(str)
test_df = delete_wrong_dim(test_df)

In [ ]:
class CBIRDataset(Dataset):
    def __init__(self, dataFrame):
        self.dataFrame = dataFrame
        
        self.transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((92, 92), antialias=True)
            ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            raise NotImplementedError('slicing is not supported')
        
        row = self.dataFrame.iloc[key]
        image = self.transformations(Image.open(row['image']).convert('RGB'))
        return image
    
    def __len__(self):
        return len(self.dataFrame.index)

In [ ]:
# Intermediate Function to process data from the data retrival class
def prepare_data(trainDF,validateDF,testDF):
    train_set = CBIRDataset(trainDF)
    validate_set = CBIRDataset(validateDF)
    test_set = CBIRDataset(testDF)
    return train_set, validate_set, test_set

In [ ]:
train_set, validate_set, test_set = prepare_data(train_df, val_df, test_df) #call CBIRDataset

## AutoEncoder

In [ ]:
class ConvAutoencoderB(nn.Module):
    def __init__(self):
        super(ConvAutoencoderB, self).__init__()
        self.encoder = nn.Sequential(# in- (N,3,92,92)
            # M = floor(M - K + 2*P)/S + 1 = 92 - 3 + 2 /2  + 1 = 46
            nn.Conv2d(in_channels=3, 
                      out_channels=64, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (32,16,46,46)
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (N,16,23,23)  46 - 2  /2 + 1= 23
            
            nn.Conv2d(in_channels=64, 
                      out_channels=32, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (N,8,43,43) 23 -  3 + 2  /  2 + 1 
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (N,8,11,11) 12-2 /1 + 1 = 11
        )
        self.decoder = nn.Sequential(
            #strides * (input_size-1) + kernel_size - 2*padding
            nn.ConvTranspose2d(in_channels = 32, 
                               out_channels=16, 
                               kernel_size=(5,5), 
                               stride=2,
                              padding=1),  # (N,16,25,25) 2* (11-1) + 5 -2 = 23
            nn.BatchNorm2d(16),
            nn.ReLU(True),
 
            nn.ConvTranspose2d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=(3,3), 
                               stride=2, 
                               padding=1),  # (N,8,48,48) 2 *(23-1) + 3 - 2 = 45
            
            nn.ReLU(True),
            nn.BatchNorm2d(8),
            nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512) > 2*(45-1) + 6 -2 = 92
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class ConvAutoencoderB2(nn.Module):
    def __init__(self):
        super(ConvAutoencoderB2, self).__init__()
        self.encoder = nn.Sequential(# in- 92
            
            nn.Conv2d(in_channels=3, 
                      out_channels=64, 
                      kernel_size=(3,3), 
                      stride=3, 
                      padding=1),  # 
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (N,16,85,85)
            
            nn.Conv2d(in_channels=64, 
                      out_channels=32, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (N,32,43,43)
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (N,32,42,42)
        
            
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(in_channels = 32, 
                               out_channels=16, 
                               kernel_size=(3,3), 
                               stride=2),  # (N,16,85,85)
            nn.BatchNorm2d(16),
            nn.ReLU(True),
 
            nn.ConvTranspose2d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=(5,5), 
                               stride=3, 
                               padding=1),  # (N,8,255,255)
            
            nn.ReLU(True),
            nn.BatchNorm2d(8),
            nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512)
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(# in- (N,3,512,512)
            
            nn.Conv2d(in_channels=3, 
                      out_channels=16, 
                      kernel_size=(3,3), 
                      stride=3, 
                      padding=1),  # (32,16,171,171)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (N,16,85,85)
            
            nn.Conv2d(in_channels=16, 
                      out_channels=8, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (N,8,43,43)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (N,8,42,42)
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(in_channels = 8, 
                               out_channels=16, 
                               kernel_size=(3,3), 
                               stride=2),  # (N,16,85,85)
            nn.ReLU(True),
 
            nn.ConvTranspose2d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=(5,5), 
                               stride=3, 
                               padding=1),  # (N,8,255,255)
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512)
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# Training

In [ ]:
#training losses

def load_ckpt(checkpoint_fpath, model, optimizer):
    
    # load check point
    checkpoint = torch.load(checkpoint_fpath)

    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['model_state_dict'])

    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # initialize valid_loss_min from checkpoint to valid_loss_min
    #valid_loss_min = checkpoint['valid_loss_min']

    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch']

def save_checkpoint(state, filename):
    """Save checkpoint if a new best is achieved"""
    print ("=> Saving a new best")
    torch.save(state, filename)  # save checkpoint


def train_model(model,  
                criterion, 
                optimizer, 
                num_epochs):
    
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
   
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            
            # Iterate over data.
            for idx,inputs in enumerate(Bar(dataloaders[phase])): 
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    loss = criterion(outputs, inputs)
        
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # statistics
                running_loss += loss.item() * inputs.size(0)
           
            epoch_loss = running_loss / dataset_sizes[phase]
            if phase == 'train':
                train_cost.append(epoch_loss)
            else:
                val_cost.append(epoch_loss)
                
                     
            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                save_checkpoint(state={   
                                    'epoch': epoch,
                                    'state_dict': model.state_dict(),
                                    'best_loss': best_loss,
                                    'optimizer_state_dict':optimizer.state_dict()
                                },filename='./result/ckpt_epoch_{}.pt'.format(epoch))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, optimizer, epoch_loss

In [ ]:
EPOCHS = 20
NUM_BATCHES = 32
RETRAIN = False

dataloaders = {'train': DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=0),
                'val':DataLoader(validate_set, batch_size=NUM_BATCHES, num_workers=0)
                }

dataset_sizes = {'train': len(train_set),'val':len(validate_set)}
              
model = ConvAutoencoderB().to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
train_cost=[]
val_cost=[]

In [ ]:
model, optimizer, loss = train_model(model=model, 
                    criterion=criterion, 
                    optimizer=optimizer, 
                    num_epochs=EPOCHS)

In [ ]:
# Plot the loss and accuracy
plt.plot(train_cost)
plt.plot(val_cost)
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:

plt.figure(figsize=(20, 4))
print("Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    pic = test_set[i].numpy().transpose((1,2,0))
    pic = (pic-np.min(pic)) / (np.max(pic)-np.min(pic))
    plt.imshow(pic)
plt.show()

# model(test_set[0])
plt.figure(figsize=(20, 4))
print("Reconstruction of Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    pic = model(test_set[i].unsqueeze(0)).squeeze(0).detach().numpy().transpose((1,2,0))
    pic = (pic-np.min(pic)) / (np.max(pic)-np.min(pic))
    plt.imshow(pic)
plt.show()

    

In [ ]:
# Save the Trained Model
torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, './cbirpretrained/conv_autoencoderBatchNorm_B_50.pt')

In [ ]:
# If re-training is required:
# Load the old model
if RETRAIN == True:
    load the saved checkpoint
    model, optimizer, start_epoch = load_ckpt('./cbirpretrained/conv_autoencoderBatchNorm_B_20.pt', model, optimizer)
    print('Checkpoint Loaded')

# Metrics

In [ ]:
size = 92
transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((size, size), antialias=True),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

In [ ]:
# Load Model in Evaluation phase
model = ConvAutoencoderB().to(device)
#replace the model's parameters with those of the trained net
model.load_state_dict(torch.load('./cbirpretrained/conv_autoencoderBatchNorm_B_20.pt', map_location=device)['model_state_dict'], strict=False)
model.eval()

In [ ]:
#get laten features = take the output encoder part of the pretrained model 
num_img = len(train_df) 
out_dim = 11
num_output = 32
def get_latent_features(images, transformations):
    latent_features = np.zeros((num_img,num_output,out_dim,out_dim)) # M in last layer of encoder()
    
    for i,image in enumerate(tqdm(images)):
        tensor = transformations(Image.open(image).convert('RGB')).to(device)
        latent_features[i] = model.encoder(tensor.unsqueeze(0)).cpu().detach().numpy()
        
    del tensor
    gc.collect()
    return latent_features

In [ ]:
images = train_df.image.values
latent_feature = get_latent_features(images, transformations)
print(images.shape)
print(latent_feature.shape)

In [ ]:
m = nn.AdaptiveAvgPool2d(1)
latent_feature = torch.tensor(latent_feature)
latent_feature.shape
pred = m(latent_feature)

In [ ]:
pred= pred.view(pred.shape[:2])
pred.shape

In [ ]:
from sklearn.cluster import KMeans
classifier = KMeans(n_clusters=39, random_state=0).fit(pred)
classifier.labels_.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score
nmis = normalized_mutual_info_score(y_train, classifier.labels_)
nmis

In [ ]:
from scipy.special import comb
def rand_index_score(clusters, classes):
    tp_plus_fp = comb(np.bincount(clusters), 2).sum()
    tp_plus_fn = comb(np.bincount(classes), 2).sum()
    A = np.c_[(clusters, classes)]
    tp = sum(comb(np.bincount(A[A[:, 0] == i, 1]), 2).sum()
             for i in set(clusters))
    fp = tp_plus_fp - tp
    fn = tp_plus_fn - tp
    tn = comb(len(A), 2) - tp - fp - fn
    return (tp + tn) / (tp + fp + fn + tn)

In [ ]:
ri = rand_index_score (y_train, classifier.labels_)
ri

# 2. Image Retrieval

In [ ]:
num_img = len(train_df) + len(val_df)
out_dim = 11
num_output = 32
def get_latent_features(images, transformations):
    latent_features = np.zeros((num_img,num_output,out_dim,out_dim)) # M in last layer of encoder()
    
    for i,image in enumerate(tqdm(images)):
        tensor = transformations(Image.open(image).convert('RGB')).to(device)
        latent_features[i] = model.encoder(tensor.unsqueeze(0)).cpu().detach().numpy()
        
    del tensor
    gc.collect()
    return latent_features

In [ ]:
df = pd.concat([train_df, val_df])
images = df.image.values
latent_features = get_latent_features(images, transformations)

In [ ]:
indexes = list(range(0, num_img))
feature_dict = dict(zip(indexes,latent_features))
index_dict = {'indexes':indexes,'features':latent_features}

In [ ]:
def euclidean(a, b):
    # compute and return the euclidean distance between two vectors
    return np.linalg.norm(a - b)

In [ ]:
def perform_search(queryFeatures, index, maxResults=64):

    results = []

    for i in range(0, len(index["features"])):
        # compute the euclidean distance between our query features
        # and the features for the current image in our index, then
        # update our results list with a 2-tuple consisting of the
        # computed distance and the index of the image
        d = euclidean(queryFeatures, index["features"][i])
        results.append((d, i))
    
    # sort the results and grab the top ones
    results = sorted(results)[:maxResults]
    # return the list of results
    return results

In [ ]:
# take the features for the current image, find all similar
# images in our dataset, and then initialize our list of result
queryIdx =2000 # Input Index for which images #800, 2000, 5490,6789
MAX_RESULTS = 5

queryFeatures = latent_features[queryIdx]
results = perform_search(queryFeatures, index_dict, maxResults=MAX_RESULTS+1)
imgs = []
img_name = []
# loop over the results
for (d, j) in results:
    if j != queryIdx:
        img = np.array(Image.open(images[j]))
        img_name.append(j)
        imgs.append(img)


fig = plt.figure(figsize=(10,5))
gs = fig.add_gridspec(2,MAX_RESULTS)

# display the query image
#plt.gca().set_title('query image')
ax0 = fig.add_subplot(gs[0,:])

plt.axis('off')
plt.title('Query Image idx_' + str(queryIdx))
ax0.imshow(np.array(Image.open(images[queryIdx])))

for i in range(MAX_RESULTS):
    ax = fig.add_subplot(gs[1, i])
    plt.title('idx_' + str(img_name[i]))
    plt.axis('off')
    ax.imshow(imgs[i])
plt.show()

In [ ]:
num_img = len(test_df)
testimages = test_df.image.values
test_latent_features = get_latent_features(testimages, transformations)
indexes = list(range(0, num_img))
feature_dict = dict(zip(indexes,test_latent_features))
index_dict = {'indexes':indexes,'features':test_latent_features}

In [ ]:
#fig, ax = plt.subplots(2,figsize=(15,15))
MAX_RESULTS = 5
queryIdx = 34 #between 0 and 65

queryFeatures = test_latent_features[queryIdx]
results = perform_search(queryFeatures, index_dict, maxResults=MAX_RESULTS+1)
imgs = []
img_name = []

# loop over the results
for (d, j) in results:
    if j != queryIdx:
        img = np.array(Image.open(testimages[j]))
        img_name.append(j)
        imgs.append(img)

fig = plt.figure(figsize=(10, 5))

gs = fig.add_gridspec(2,5)

ax0 = fig.add_subplot(gs[0,:])
plt.axis('off')
plt.title('Query Image idx_' + str(queryIdx))
ax0.imshow(np.array(Image.open(testimages[queryIdx])))

for i in range(5):
    ax = fig.add_subplot(gs[1, i])
    plt.title('idx_' + str(img_name[i]))
    plt.axis('off')
    ax.imshow(imgs[i])
plt.show()

